## 7.1 단어 분할(Janome, MeCab+NEologd)
※ 여기 코드는 Ubuntu 환경임! Window에서랑은 동작이 다를 수 있음

### 7.1.2 머신러닝의 자연어 처리 흐름
* 코퍼스: 수집한 문서 데이터


* 머신러닝 자연어 처리 흐름
    * 데이터 청소
        * HTML 태그나 메일 헤더 등 글이 아닌 노이즈 제거
    * 정규화
        * 반각 / 전각 통일, 대소문자 통일, 표기 오류 수종, 의미 없는 숫자 대체 등
    * 단어 분할(형태소 분석)
        * 문장을 단어별로 구분
    * 기본형 변환(표제어화)
        * '달리고' → '달리다'처럼 단어를 기본형(원형)으로 변환. 기본형으로 변환하지 않는 경우도 있음
        * 경우에 따라서 스톱워드 제거
            * 스톱워드: 출현 빈도가 많은 단어나 조사, 조동사 등 문장에서 의미가 없는 품사 및 단어
    * 스톱 워드 제거
        * 출현 수가 많은 단어나 조사 등 특별한 의미가 없는 단어 제거. 제거하지 않는 경우도 있음
    * 단어의 수치화
        * 단어를 기계학습응로 다룰 수 있도록 수치 변경. 단순히 ID를 매기거나 벡터로 표현할 수 있음

### 7.1.3 Janome를 활용한 단어 분할
* 실행결과: **문장이 단어로 분리**되고 **품사 정보 등이 표시**됨

In [1]:
from janome.tokenizer import Tokenizer

j_t = Tokenizer()

text = '機械学習が好きです。'

for token in j_t.tokenize(text):
    print(token)

機械	名詞,一般,*,*,*,*,機械,キカイ,キカイ
学習	名詞,サ変接続,*,*,*,*,学習,ガクシュウ,ガクシュー
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
好き	名詞,形容動詞語幹,*,*,*,*,好き,スキ,スキ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
。	記号,句点,*,*,*,*,。,。,。


* `j_t.tokenize.wakati = True`: Janome에서 단어 분할만 필요하고 단어의 품사 정보 등이 필요 없을 때 설정!

In [2]:
# 단어 분할 함수를 정의
def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text, wakati=True)]


text = '機械学習が好きです。'
print(tokenizer_janome(text))


['機械', '学習', 'が', '好き', 'です', '。']


### 7.1.4 MeCab + NEologd에 의한 단어 분할
* `MeCab`
    * 단어 분할 라이브러리
    * 신조어 사전 `NEologd`와 함께 사용되는 경우가 많음
    * 신조어에 대응할 수 있음

* 형태소 분석 실행

In [3]:
import MeCab

m_t = MeCab.Tagger('-Ochasen')

text = '機械学習が好きです。'

print(m_t.parse(text))


機械	キカイ	機械	名詞-一般		
学習	ガクシュウ	学習	名詞-サ変接続		
が	ガ	が	助詞-格助詞-一般		
好き	スキ	好き	名詞-形容動詞語幹		
です	デス	です	助動詞	特殊・デス	基本形
。	。	。	記号-句点		
EOS



In [4]:
import MeCab

m_t = MeCab.Tagger('-Ochasen -d /usr/lib/mecab/dic/mecab-ipadic-neologd')

text = '機械学習が好きです。'

print(m_t.parse(text))


機械学習	キカイガクシュウ	機械学習	名詞-固有名詞-一般		
が	ガ	が	助詞-格助詞-一般		
好き	スキ	好き	名詞-形容動詞語幹		
です	デス	です	助動詞	特殊・デス	基本形
。	。	。	記号-句点		
EOS



* `tokenizer_mecab`
    * `Mecab`과 `NEologd`로 문장을 단어 리스트로 분할

In [5]:
# 단어 분할 함수를 정의
m_t = MeCab.Tagger('-Owakati -d /usr/lib/mecab/dic/mecab-ipadic-neologd')

def tokenizer_mecab(text):
    text = m_t.parse(text)  # これでスペースで単語が区切られる
    ret = text.strip().split()  # スペース部分で区切ったリストに変換
    return ret


text = '機械学習が好きです。'
print(tokenizer_mecab(text))


['機械学習', 'が', '好き', 'です', '。']
